In [1]:
# Re-import necessary libraries and reload the datasets
import pandas as pd

# Reload the datasets
ehr_data_path = '/content/Ops Case Study Dataset - Sample EHR Data.csv'
db_data_path = '/content/Ops Case Study Dataset - Sample DB Data.csv'

ehr_data = pd.read_csv(ehr_data_path)
db_data = pd.read_csv(db_data_path)

# Normalize column names for consistency
ehr_data.rename(columns={'Date of Service': 'date_of_service', 'Provider Name': 'provider_name', 'CPT Code': 'cpt_code'}, inplace=True)
db_data.rename(columns={'from_date_range': 'date_of_service', 'Provider Name': 'provider_name', 'cpt_codes': 'cpt_code'}, inplace=True)

# Convert date formats to standard datetime format
ehr_data['date_of_service'] = pd.to_datetime(ehr_data['date_of_service'], errors='coerce').dt.date
db_data['date_of_service'] = pd.to_datetime(db_data['date_of_service'], errors='coerce').dt.date

# Create unique encounter IDs
ehr_data['encounter_id'] = ehr_data['Patient Name'] + '|' + ehr_data['provider_name'] + '|' + ehr_data['date_of_service'].astype(str)
db_data['encounter_id'] = db_data['Patient Name'] + '|' + db_data['provider_name'] + '|' + db_data['date_of_service'].astype(str)

# Check for duplicates and NaN values in encounter_id columns
ehr_duplicates = ehr_data['encounter_id'].duplicated().sum()
db_duplicates = db_data['encounter_id'].duplicated().sum()
print("duplicates=>", ehr_duplicates, db_duplicates)

ehr_nan = ehr_data['encounter_id'].isna().sum()
db_nan = db_data['encounter_id'].isna().sum()
print("null=>", ehr_nan, db_nan)
# Clean data: Remove duplicates and NaN values
ehr_cleaned = ehr_data.dropna(subset=['encounter_id']).drop_duplicates(subset=['encounter_id'])
db_cleaned = db_data.dropna(subset=['encounter_id']).drop_duplicates(subset=['encounter_id'])

# Identify missing encounters after cleaning
missing_encounters_cleaned = ehr_cleaned[~ehr_cleaned['encounter_id'].isin(db_cleaned['encounter_id'])]

# Output summary
ehr_duplicates, db_duplicates, ehr_nan, db_nan, missing_encounters_cleaned.shape


duplicates=> 19343 108
null=> 2 0


(19343, 108, 2, 0, (282, 5))

In [2]:
missing_encounters_cleaned

,Patient Name,provider_name,date_of_service,cpt_code,encounter_id
22,Olivia Miller,Sebastian Miller,2024-07-23,97140,Olivia Miller|Sebastian Miller|2024-07-23
30,Noah Gonzalez,Elijah Johnson,2024-08-01,97140,Noah Gonzalez|Elijah Johnson|2024-08-01
72,Julian King,Noah Lee,2024-09-11,97112,Julian King|Noah Lee|2024-09-11
76,Layla Miller,Aiden King,2024-08-14,97140,Layla Miller|Aiden King|2024-08-14
83,Sebastian Jackson,Sebastian Miller,2024-09-26,TOS115,Sebastian Jackson|Sebastian Miller|2024-09-26
...,...,...,...,...,...
16026,Mason Turner,Sebastian Martinez,2024-09-24,97140,Mason Turner|Sebastian Martinez|2024-09-24
17614,Mia Brown,Charlotte Williams,2024-09-19,97110,Mia Brown|Charlotte Williams|2024-09-19
18681,Ethan Smith,Sebastian Martinez,2024-07-03,NORCM,Ethan Smith|Sebastian Martinez|2024-07-03
19779,Ava Hernandez,Noah Lee,2024-09-26,97110,Ava Hernandez|Noah Lee|2024-09-26


In [ ]:
missing_encounters_cleaned.to_csv('missing_encounters.csv')

In [4]:
# Step 1: Clean and structure the data for the list of missing encounters
missing_encounters_summary = missing_encounters_cleaned.sort_values(by=['date_of_service', 'provider_name'])

# Step 2: Analyze trends in the data
provider_trends = missing_encounters_cleaned['provider_name'].value_counts()
cpt_code_trends = missing_encounters_cleaned['cpt_code'].value_counts()
date_trends = missing_encounters_cleaned['date_of_service'].dt.month.value_counts().sort_index()


# Prepare the summary for the deliverables
summary_output = {
    "provider_trends": provider_trends.head(10),  # Top 10 providers with missing encounters
    "cpt_code_trends": cpt_code_trends.head(10),  # Top 10 CPT codes for missing encounters
    "date_trends": date_trends,  # Distribution by month
}


summary_output


<ipython-input-4-c9d939c7d611>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  missing_encounters_cleaned['date_of_service'] = pd.to_datetime(missing_encounters_cleaned['date_of_service'], errors='coerce')


{'provider_trends': provider_name
 Aiden King            95
 Noah Lee              52
 Sebastian Miller      37
 Liam Young            36
 Charlotte Williams    20
 Sebastian Martinez    19
 Julian Lee            13
 Elijah Johnson        10
 Name: count, dtype: int64,
 'cpt_code_trends': cpt_code
 97140     67
 97110     59
 NORCM     47
 97010     31
 97112     29
 97014     15
 sp90       5
 G0283      4
 TOS115     4
 SP         3
 Name: count, dtype: int64,
 'date_trends': date_of_service
 7     65
 8    106
 9    111
 Name: count, dtype: int64}